In [1]:
import os
import pandas as pd
from tqdm import tqdm

from androguard.core.bytecodes.apk import APK

In [2]:
BASE_DATASET = r"C:\Users\SHAIKH\OneDrive\Desktop\ML\ML\dataset"   # <-- YOUR PATH
BENIGN_DIR = os.path.join(BASE_DATASET, "benign")
MALWARE_DIR = os.path.join(BASE_DATASET, "malware")

print("Benign folder:", BENIGN_DIR)
print("Malware folder:", MALWARE_DIR)

Benign folder: C:\Users\SHAIKH\OneDrive\Desktop\ML\ML\dataset\benign
Malware folder: C:\Users\SHAIKH\OneDrive\Desktop\ML\ML\dataset\malware


In [3]:
def extract_permissions(apk_path):
    try:
        apk = APK(apk_path)
        perms = apk.get_permissions()
        clean = [p.split(".")[-1] for p in perms]
        return clean
    except Exception as e:
        print("Error:", e)
        return []


In [4]:
apk_records = []

print("\n📦 Extracting from BENIGN APKs...")
for file in tqdm(os.listdir(BENIGN_DIR)):
    if file.endswith(".apk"):
        path = os.path.join(BENIGN_DIR, file)
        perms = extract_permissions(path)
        apk_records.append([file, perms, 0])   # 0 = benign


print("\n⚠️ Extracting from MALWARE APKs...")
for file in tqdm(os.listdir(MALWARE_DIR)):
    if file.endswith(".apk"):
        path = os.path.join(MALWARE_DIR, file)
        perms = extract_permissions(path)
        apk_records.append([file, perms, 1])   # 1 = malware


df = pd.DataFrame(apk_records, columns=["apk_name", "permissions", "label"])
print("\nSample data:")
display(df.head())



📦 Extracting from BENIGN APKs...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.34it/s]



⚠️ Extracting from MALWARE APKs...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 11.83it/s]



Sample data:


,apk_name,permissions,label
0,ChatGPT[1].apk,"[FOREGROUND_SERVICE_MEDIA_PROJECTION, USE_BIOM...",0
1,Google_Play_Store[1].apk,"[WRITE_APP_PROVIDER, RUN_DOCK_SETUP, SEND_DEVI...",0
2,WhatsApp[1].apk,"[FOREGROUND_SERVICE_DATA_SYNC, FOREGROUND_SERV...",0
3,YouTube[1].apk,"[FOREGROUND_SERVICE_DATA_SYNC, FOREGROUND_SERV...",0
4,LuckyPatchers.com_Official_Installer_11.9.6[1]...,"[VIBRATE, USE_COMPONENT, FOREGROUND_SERVICE, A...",1


In [5]:
print("\n🔧 Building permission vocabulary...")

all_perms = set()
for perm_list in df["permissions"]:
    all_perms.update(perm_list)

permission_columns = sorted(list(all_perms))

print("Total unique permissions:", len(permission_columns))




🔧 Building permission vocabulary...
Total unique permissions: 184


In [6]:
print("\n⚙️ Converting permissions → ML feature rows...")

feature_rows = []

for _, row in df.iterrows():
    features = [1 if p in row["permissions"] else 0 for p in permission_columns]
    features.append(row["label"])  # add label
    feature_rows.append(features)

df_features = pd.DataFrame(feature_rows, columns=permission_columns + ["label"])

print("\nFeature preview:")
display(df_features.head())



⚙️ Converting permissions → ML feature rows...

Feature preview:


,ACCESS_ADSERVICES_AD_ID,ACCESS_ADSERVICES_ATTRIBUTION,ACCESS_COARSE_LOCATION,ACCESS_DOWNLOAD_MANAGER,ACCESS_DOWNLOAD_MANAGER_ADVANCED,ACCESS_FINE_LOCATION,ACCESS_INSTANT_APPS,ACCESS_MEDIA_LOCATION,ACCESS_NETWORK_STATE,ACCESS_NOTIFICATION_POLICY,...,WRITE_CONTACTS,WRITE_DEVICE_CONFIG,WRITE_EXTERNAL_STORAGE,WRITE_GSERVICES,WRITE_MEDIA_STORAGE,WRITE_SECURE_SETTINGS,WRITE_SETTINGS,WRITE_SYNC_SETTINGS,WRITE_VERIFY_APPS_CONSENT,label
0,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,0,1,0,1,1,...,0,1,1,1,0,1,1,0,1,0
2,0,0,1,0,0,1,0,1,1,0,...,1,0,1,0,0,0,1,1,0,0
3,1,1,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,1


In [7]:
output_path = os.path.join(BASE_DATASET, "extracted_permission.csv")
df_features.to_csv(output_path, index=False)

print("\n🎉 DONE! ML-Ready file saved at:")
print(output_path)


🎉 DONE! ML-Ready file saved at:
C:\Users\SHAIKH\OneDrive\Desktop\ML\ML\dataset\extracted_permission.csv


In [8]:
#5. Save the permission column list for prediction
# ==============================================
import pickle

perm_path = os.path.join(BASE_DATASET, "permission_columns.pkl")
with open(perm_path, "wb") as f:
    pickle.dump(permission_columns, f)

print("📝 Saved permission column list at:", perm_path)

📝 Saved permission column list at: C:\Users\SHAIKH\OneDrive\Desktop\ML\ML\dataset\permission_columns.pkl
